In [4]:
#导入库+包
import re
import os
import fitz
import xlwings as xw
from PIL import Image

In [ ]:
#前提设置
dimlimit = 600  # each image side must be greater than this


In [5]:
# 打开所需文件
path = r"D:\study\Python\jupyter\ImagesConducted\NascionActicles\253083-gao2016.pdf" #导入的pdf的路径
pic_path = r"D:\study\Python\jupyter\ImagesConducted\images" #导出图片的文件夹
if not os.path.exists(pic_path):
    os.mkdir(pic_path)
else:
    print("文件夹已存在，不必重建！")

doc = fitz.open(path) #打开pdf文件
lenXREF = doc.xrefLength()-1 #获取对象数量长度
print(f"文件名:{path}, 页数: {doc.pageCount}, 对象: {lenXREF}") # 打印PDF的信息

Document('D:\study\Python\jupyter\ImagesConducted\NascionActicles\193217-Kamali-2015-Pressure-induced-phase-transformati.pdf')


In [6]:
#从文件中提取doi进入表中并给sheet命名
def DOIsearch():
    text0=doc[0].getText()
#     print(text)
#     pat=re.compile(r'^doi',re.I|re.M)
#     doi=re.search('DOI|doi',text)
#     doimbr=re.match('\b(10[.][0-9]{4,}*/[A-Z]0036023606080043',text)
    doimbr=re.search('10[.][0-9]{4,}/[A-Z,a-z,.,0-9,-,|, ]{,}',text)
    sheet.range('A1').value=DOI
    sheet.range('B1'.value=doimbr)
#     print(doi)
#     print(doimbr)
    sheet.range('B1').value

<re.Match object; span=(4995, 5021), match='10.1016/j.jssc.2014.10.017'>


In [2]:
#从文件中逐页提取图片标签进入表中
def FIGsearch1():
    line=1
    for page in doc:
        text=page.getText()
    #     print(text)
        fig=re.findall('[\n,\t]fig\. [0-9]{1,}\. .{1,300}\n',text,re.I|re.S)
    #     fig=re.findall('[Figure,[Ff]ig]',text)
        for i in fig:
            sheet.range('C%s'%(line)).value

In [ ]:
#从文件中逐页提取图片标签及其描述方式进入表中
def FIGsearch2():
    for i in range(1,line):
        for page in doc:
            text=page.getText()
            fig=re.findall('.{1,200}fig\. %s.{1,200}\n'%(i),text,re.I|re.S)
            sheet.range('D%s'%(i)).value
            continue

In [ ]:
#从文档中提取并过滤图片放入文件夹
def extractimages():
    checkIM = r"/Subtype(?= */Image)" # 使用正则表达式来查找图片
#     for i in range(1, lenXREF):
#         text = doc.xrefObject(i) # 定义对象字符串       
#         isImage = re.search(checkIM, text) # 查看是否是image
#         if isImage:
#             print(i)
    imgcount = 0  # 图片计数
    for i in range(1, lenXREF):
        text = doc.xrefObject(i) # 定义对象字符串       
        isImage = re.search(checkIM, text) # 查看是否是image
        if not isImage: # 如果不是图片，则进行下一个判断
            continue 

        pix = fitz.Pixmap(doc, i) # 生成图像对象    
        print(pix)

        if min(pix.width, pix.height) <= dimlimit:  # rectangle edges too small
    #         print("CCC")
            continue

    #     imgdata = imgdict["image"]  # image data
    #     l_imgdata = len(imgdata)  # length of data
    #     if l_imgdata <= abssize:  # image too small to be relevant
    #         continue
        try:
            if pix.n<5:
                imgcount += 1 #图片计数
                new_name = f"图片{imgcount}.png" # 生成图片的名称
                pix.writePNG(os.path.join(pic_path, new_name))
                print(new_name)
                pix = None
            else:
                imgcount += 1 #图片计数
                pixno = fitz.Pixmap(fitz.csRGB, pix) 
                new_name = f"图片{imgcount}.png" # 生成图片的名称
                pixno.writePNG(os.path.join(pic_path, new_name))
                print(new_name)
                pixno = None
        except RuntimeError:
            #异常处理，跳过错误段继续提取之后的图片
            print("Error:extract image fail")
            os.remove(os.path.join(pic_path, new_name))
            imgcount-=1
            continue


#     for i in range(len(doc)):
#         for img in doc.getPageImageList(i):
#         #识别并获取文献中的图片对象
#             try:
#                 xref=img[0]
#                 pix=fitz.Pixmap(doc,xref)
#                 if pix.n <5:
#                     pix.writePNG(os.path.join(picPath,"%s%s Page NO.%s.png"%(picName,i,num)))
#                     #将图片存储为PNG格式
#                     num+=1
#                     pix=None
#                 else:
#                     pixno = fitz.Pixmap(fitz.csRGB, pix) 
#                     #若非RGB图片，首先转换为RBG图片再作PNG格式保存
#                     pixno.writePNG(os.path.join(picPath,"%s%s Page NO.%s.png"%(picName,i,num)))
#                     num+=1
#                     pixno=None
#                     #清空当前图形对象
#             except RuntimeError:
#             #异常处理，跳过错误段继续提取之后的图片
#                 print("Error:extract image fail when dealing with {}".format(picName))
#                 continue

In [ ]:
#将图片放入到sheet中
def add_center(sht, target, filePath, match=False, width=None, height=None, column_width=None, row_height=None):
    '''Excel智能居中插入图片

    优先级：match > width & height > column_width & row_height
    建议使用column_width或row_height，定义单元格最大宽或高

    :param sht: 工作表
    :param target: 目标单元格，字符串，如'A1'
    :param filePath: 图片绝对路径
    :param width: 图片宽度
    :param height: 图片高度
    :param column_width: 单元格最大宽度，默认100像素，0 <= column_width <= 1557.285
    :param row_height: 单元格最大高度，默认75像素，0 <= row_height <= 409.5
    :param match: 绝对匹配原图宽高，最大宽度1557.285，最大高度409.5
    '''
    unit_width = 6.107  # Excel默认列宽与像素的比
    rng = sht.range(target)  # 目标单元格
    name = os.path.basename(filePath)  # 文件名
    _width, _height = Image.open(filePath).size  # 原图片宽高
    NOT_SET = True  # 未设置单元格宽高
    # match
    if match:  # 绝对匹配图像
        width, height = _width, _height
    else:  # 不绝对匹配图像
        # width & height
        if width or height:
            if not height:  # 指定了宽，等比计算高
                height = width / _width * _height
            if not width:  # 指定了高，等比计算宽
                width = height / _height * _width
        else:
            # column_width & row_height
            if column_width and row_height:  # 同时指定单元格最大宽高
                width = row_height / _height * _width  # 根据单元格最大高度假设宽
                height = column_width / _width * _height  # 根据单元格最大宽度假设高
                area_width = column_width * height  # 假设宽优先的面积
                area_height = row_height * width  # 假设高优先的面积
                if area_width > area_height:
                    width = column_width
                else:
                    height = row_height
            elif not column_width and not row_height:  # 均无指定单元格最大宽高
                column_width = 100
                row_height = 75
                rng.column_width = column_width / unit_width  # 更新当前宽度
                rng.row_height = row_height  # 更新当前高度
                NOT_SET = False
                width = row_height / _height * _width  # 根据单元格最大高度假设宽
                height = column_width / _width * _height  # 根据单元格最大宽度假设高
                area_width = column_width * height  # 假设宽优先的面积
                area_height = row_height * width  # 假设高优先的面积
                if area_width > area_height:
                    height = row_height
                else:
                    width = column_width
            else:
                width = row_height / _height * _width if row_height else column_width  # 仅设了单元格最大宽度
                height = column_width / _width * _height if column_width else row_height  # 仅设了单元格最大高度
    assert 0 <= width / unit_width <= 255
    assert 0 <= height <= 409.5
    if NOT_SET:
        rng.column_width = width / unit_width  # 更新当前宽度
        rng.row_height = height  # 更新当前高度
    left = rng.left + (rng.width - width) / 2  # 居中
    top = rng.top + (rng.height - height) / 2
    try:
        sht.pictures.add(filePath, left=left, top=top, width=width, height=height, scale=None, name=name)
    except Exception:  # 已有同名图片，采用默认命名
        pass